In [1]:
import torch
import numpy as np
import torch.nn as nn
import os
import sys

In [2]:
notebook_path = os.getcwd()
parent_dir = os.path.dirname(notebook_path)
sys.path.append(parent_dir)

In [3]:
import importnb
with __import__('importnb').Notebook(): 
    from tools import ScaledDotProductAttention
    from tools import MultiHeadAttention
    from tools import AddPositionalEncoding
    from tools import TransformerFFN
    from notebooks.model import TransformerEncoderLayer,TransformerEncoder
    from notebooks.model import TransformerDecoderLayer

## SelfAttentionの実行

### 適当なデータの作成
---
新しくデータkを作る。kは**(B,T,d_model)**のshapeを持つテンソルである。


### SelfAttentionにする
---
q,k,vを同じテンソルにすることでSelfAttentionにする。


### forwardで計算を行う。
---
呼び出したspa.forward()によって計算を行う。
この時spaの初期化で与えるd_kには本来d_modelをhead数で割った値が入る(デフォルトだと512/8で64)

In [12]:
###SelfAttentionの実行
num_head = 4
batch_size = 2
seq_len = 8
d_model = 64
max_len = 512
d_ff =1024
N = 6
pad_idx = 0
vocab_size = 4200
dropout_rate=0.1
layer_norm_eps = 1e-5
q = torch.randint(0,2000, (batch_size, seq_len)).to(torch.int64)
k = torch.randint(0,2000, (batch_size, seq_len)).to(torch.int64)
v = k

## MultiHeadAttentionの実行
ランダムなテンソル、スタンダードなマスクを使う。

In [8]:
def create_incremental_mask(seq_len):
    """
    seq_len x seq_len のサイズのマスクを生成する。
    0列目は全てFalse、以降の列では上から順にTrueの数を増やしていく。
    """
    # seq_len x seq_len の行列を生成し、初期値は全てFalseに設定
    mask = torch.full((seq_len, seq_len), False)

    # 各列に対して、上から順にTrueをセットする
    for i in range(seq_len):
        mask[:i, i] = True

    return mask
self_mask = create_incremental_mask(seq_len).repeat(batch_size,1,1)
tgt_mask = create_incremental_mask(seq_len).repeat(batch_size,1,1)
print(self_mask.shape)
print(tgt_mask.shape)

torch.Size([2, 8, 8])
torch.Size([2, 8, 8])


In [9]:
encoder = TransformerEncoder(
    d_model = d_model,
    d_ff = d_ff,
    num_head = num_head,
    N=N,
    max_len=max_len,
    pad_idx=pad_idx,
    vocab_size=vocab_size,
    dropout_rate=dropout_rate,
    layer_norm_eps=layer_norm_eps
)

In [11]:
output = encoder(
    k.to(torch.int32),
    mask=self_mask)

torch.Size([2, 8, 64])


In [14]:
embed = nn.Embedding(2000,d_model,pad_idx)
q = embed(q)
k = embed(k)
v = embed(v)

In [15]:
q.shape

torch.Size([2, 8, 64])

In [16]:
decoderlayer = TransformerDecoderLayer(
    d_model=d_model,
    d_ff=d_ff,
    num_head=num_head,
    dropout_rate=dropout_rate,
    layer_norm_eps=layer_norm_eps
)

In [17]:
output = decoderlayer(
    q,
    k,
    tgt_mask,
    self_mask
)

In [19]:
output.shape

torch.Size([2, 8, 64])